# Sentinel 2 Cloudless Mosaic

This notebook selects and crops a *cloudless* yearly time series of [Sentinel-2 Level-2A](https://planetarycomputer.microsoft.com/dataset/sentinel-2-l2a) images and is modified from the example notebook provided by Microsoft. This notebook performs the following steps:

* Find a time series of images within a bounding box
* Remove images that contain clouds
* Isolate the near-infrared (NIR) band (band 8 for Sentinel-2)
* Select a set of annual images that occur on approximately the same day of the year
* Save the results to GeoTiffs

## Setup

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import xarray as xr
import pandas as pd

import rasterio
import rasterio.features
import rioxarray
import stackstac
import pystac_client
import planetary_computer

import pyproj
from shapely.ops import transform
from shapely.geometry import Polygon

import xrspatial.multispectral as ms

import dask
from dask_gateway import GatewayCluster
from dask import visualize

import itertools
from datetime import datetime
from tqdm.notebook import tqdm

import geopandas as gpd
from pathlib import Path
from datetime import datetime
from shapely.geometry import shape,box
from skimage.morphology import dilation

## Helpful functions

In [ ]:
def polygon_to_raster(gdf,template_path,value=1,crs=False):
    if isinstance(gdf,str):
        pol = gpd.read_file(gdf)
    else:
        pol = gdf

    with rasterio.open(template_path) as dst:
        profile = dst.profile
        template_crs = dst.crs
        template_transform = profile['transform']
        template_shape = dst.shape

    # if crs != pol.crs:
    #   raise Exception('CRSs do not match!')

    geojsons = [x['geometry'] for x in pol.geometry.__geo_interface__['features']]
    if isinstance(value,str):
        shapes = [tuple(x) for x in zip(geojsons,pol[value])]
    else:
        shapes = [(x,value) for x in geojsons]

    array = rasterio.features.rasterize(shapes, out_shape=template_shape, transform=template_transform)
    
    result = [array, profile]
    if crs:
        result.append(template_crs)

    return result


def write_raster(array,profile,out_path,nodata,dtype):
    # From rasterio docs:
    # Register GDAL format drivers and configuration options with a
    # context manager.
    with rasterio.Env():
        # And then change the band count to 1, set the
        # dtype to uint8, and specify LZW compression.
        profile.update(
            dtype=dtype,
            count=1,
            nodata=nodata,
            compress='lzw')

        with rasterio.open(out_path, 'w', **profile) as dst:
            dst.write(array.astype(dtype), 1)

    return out_path

## Create a Dask cluster

We're going to process a large amount of data. To cut down on the execution time, we'll use a Dask cluster to do the computation in parallel, adaptively scaling to add and remove workers as needed. See [Scale With Dask](../quickstarts/scale-with-dask.ipynb) for more on using Dask.

In [ ]:
# Set up the cluster
cluster = GatewayCluster()  # Creates the Dask Scheduler. Might take a minute.
client = cluster.get_client()
cluster.adapt(minimum=4, maximum=32)

In [ ]:
print(cluster.dashboard_link)

## Discover data

In this step we define our bounding box by creating a Shapely Polygon object. The Polygon object is created from a set of coordinate pairs in **Latitude and Longitude** (epsg 3857). A simple way of getting the coordinate pairs is by creating a bounding box in Google Earth, saving it to a kml, then opening it as a text file and copying the coordinates.

At this point, you'll have to decide if you want to process multiple years at once, or if you want to process the years separately. This decision comes down to how much compute power you have access to. For the Dask cluster parameters specified, `cluster.adapt(minimum=4, maximum=24)`, the maximum amount of images used should be less than 200. You can alter the number of images used by changing the values of `date_range`, `max_cloud`, and `pol`.

In [ ]:
#proroa: R129, T60GUA & T60HUB
#andrew: R129, T60GVA
#bird: R129, T60HUB

# setting options
date_range = '2016-01-01/2022-01-01'
local_zone = 'EPSG:32760'
buffer_width = 2000
max_cloud_image = 33
max_cloud_bbox = 1
landslide = 1

# get landslide
ls_all = gpd.read_file('landslides/landslide_sediment.shp').to_crs('EPSG:4326')
ls = ls_all.loc[ls_all['id'] == landslide]

# get bounding box (minx, miny, maxx, maxy)
bbox = tuple(ls.total_bounds)

aoi = gpd.GeoSeries([box(*bbox)],crs='EPSG:4326').to_crs(local_zone)
aoi_buffer = aoi.buffer(buffer_width)

In [ ]:
# ls.explore()

Using `pystac_client` we can search the Planetary Computer's STAC endpoint for items matching our query parameters.

In [ ]:
stac = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

search = stac.search(
    bbox=bbox,
    datetime=date_range,
    collections=["sentinel-2-l2a"],
    limit=500,  # fetch items in batches of 500
    query={"eo:cloud_cover": {"gte":0,"lte": max_cloud_image}},
)

# Get items with the correct relative orbit
items = list(search.get_items())

In [ ]:
df = pd.DataFrame()
df['orbit'] = [x.id.split('_')[3] for x in items]
df['frame'] = [x.id.split('_')[4] for x in items]
geometries = [shape(x.geometry).wkt for x in items]

gdf = gpd.GeoDataFrame(df, geometry=gpd.GeoSeries.from_wkt(geometries,crs='EPSG:4326'))
gdf['geometry'] = gdf.geometry.to_crs(local_zone)
gdf['overlap'] = gdf.geometry.intersection(aoi_buffer[0]).area / aoi_buffer[0].area
gdf = gdf[gdf['overlap'] >= 0.99]
counts = gdf.loc[:,['orbit','frame','overlap']].groupby(['orbit','frame']).count().reset_index()
orbit, frame, count = counts.loc[counts['overlap'] == counts['overlap'].max()].values[0]

image_footprint = gdf[(gdf['orbit'] == orbit) & (gdf['frame'] == frame)].iloc[:1]

print(f'Selected orbit {orbit} and frame {frame} with {count} images')

In [ ]:
m = image_footprint.explore()
m = aoi_buffer.explore(m=m,style_kwds={'color':'red'})
m = aoi.explore(m=m,style_kwds={'color':'black'})
# m

Now we restrict the results to the same orbit

In [ ]:
# grab ids
ids = np.array([x.id.split('_') for x in items])

# get correct orbit
ids = ids[ids[:,3] == orbit]

# get correct frame
ids = ids[ids[:,4] == frame]

# grab valid ids
valid_ids = ['_'.join(x) for x in ids]

#subset items
items = [x for x in items if any([y == x.id for y in valid_ids])]

print(f'Number of images before cloud masking is {len(items)}')

Depending on the location, this should return about 50-100 images for our study area over time, and cloudiness. Those items will still have *some* clouds over portions of the scenes, though. To create our cloudless mosaic, we'll load the data into an [xarray](https://xarray.pydata.org/en/stable/) DataArray using [stackstac](https://stackstac.readthedocs.io/) and then reduce the time-series of images down to a single image.

In [ ]:
signed_items = []
for item in items:
    item.clear_links()
    signed_items.append(planetary_computer.sign(item).to_dict())

## Load Data

In this step we load the data and perform some initial cleaing that includes:
* subsetting to our exact bounding box
* removing pixels that correspond clouds and clouds shadows

To perform our cloud masking, we use Sentinel-2's Scene Classification Layer ([SCL](https://sentinels.copernicus.eu/web/sentinel/technical-guides/sentinel-2-msi/level-2a/algorithm)) and mask out values 3, 8, 9, and 10.

In [ ]:
data = (
    stackstac.stack(
        signed_items,
        assets=["B08","SCL"],
        chunksize=4096*2,
        resolution=10
    )
    .where(lambda x: x > 0, other=np.nan)  # sentinel-2 uses 0 as nodata
)

# Get bounding box in projection of data
minx, miny, maxx, maxy = tuple(aoi_buffer.to_crs(data.crs).total_bounds)

# Subset data and mask clouds
data = data.sel(x=slice(minx, maxx), y=slice(maxy,miny))

## Cloud filtering

In [ ]:
first = data.groupby('time').first(skipna=False)
valid = xr.where(first.sel(band='SCL',drop=True).isin([3,8,9]),x=0,y=1)

In [ ]:
pct_valid = valid.sum(dim=['x','y']).compute().to_numpy() / (data.shape[2] * data.shape[3])
pct_valid = pct_valid.squeeze()
dates = valid.time.to_numpy()

In [ ]:
clouds = pd.DataFrame({'date':dates,'pct_valid':pct_valid[:]})

In [ ]:
best = clouds.loc[clouds.pct_valid > (100-max_cloud_bbox)/100].copy()

In [ ]:
best['year'] = best.date.dt.year
counts = best.groupby('year').count().reset_index()

In [ ]:
for i,row in counts.iterrows():
    print(f'Year {row["year"]} contains {row["date"]} images')

In [ ]:
best_dates = data.sel(band=["B08"],time=list(best.date)).squeeze()

## Closest Dates

In [ ]:
date_df = pd.DataFrame({'date':best.date})
date_df['year'] = date_df.date.dt.year
date_df['doy'] = date_df.date.dt.dayofyear
date_df['month'] = date_df.date.dt.month

In [ ]:
good_months = [9,10,11,12]
summer = date_df.loc[date_df.month.isin(good_months)].copy()
doys = [list(zip(x.date,x.doy)) for _,x in summer.groupby('year')]

In [ ]:
min_distance = np.inf

for x in tqdm(list(itertools.product(*doys))):
    distance = 0
    for d1,d2 in itertools.combinations(x,2):
        a = d1[1]
        b = d2[1]
        distance += min(abs(a-b),abs(a+365-b),abs(b-a),abs(b+365-a))

    if np.mean(distance) < min_distance:
        min_distance = np.mean(distance)
        date_list = [z[0] for z in x]

In [ ]:
closest_df = date_df.loc[date_df.date.isin(date_list)]
plt.scatter(date_df.doy,date_df.year)
plt.plot(closest_df.doy,closest_df.year,color='red')
print(min_distance,date_list)

## Check Results

In [ ]:
img_dict = {}

for t in closest_df.date:
    time_name = datetime.strftime(t,'%Y%m%d')
    name = f's2_l2_{orbit}_{frame}_{time_name}.tif'
    img_dict[name] = best_dates.sel(time=t)

In [ ]:
f, [[ax1,ax2],[ax3,ax4],[ax5,ax6]] = plt.subplots(3,2,figsize=(15,25))
axes = [ax1,ax2,ax3,ax4,ax5,ax6]
for ax, name in zip(axes,img_dict):
    ax.imshow(img_dict[name])
    ax.set_title(name)

plt.tight_layout()

## Download

In [ ]:
for name in img_dict:
    print(f'Writing {name}...')
    img_dict[name].rio.to_raster(name)

## Create active mask

In [ ]:
pols = ls_all.to_crs(local_zone)
template = list(img_dict.keys())[0]
mask, profile = polygon_to_raster(pols,template,value=1,crs=False)

In [ ]:
mask_buffered = dilation(mask,np.ones((11,11)))
plt.imshow(mask_buffered)

In [ ]:
write_raster(mask_buffered,profile,'activity_mask.tif',nodata=-32768,dtype=rasterio.int16)

## Close cluster
Once we're done with our processing, let's be a good steward of our resources and close our cluster

In [ ]:
cluster.close()

## Download Data
And you're done! The completed GeoTiff files should be in the same directory as this notebook, and can be downloaded via Jupyter's GUI